In [32]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.preprocessing import PolynomialFeatures

from surprise import Reader, Dataset, SVD
from surprise.model_selection import GridSearchCV
from surprise import AlgoBase
from surprise import Dataset
from surprise import PredictionImpossible
from surprise.model_selection import cross_validate

from tqdm import tqdm_notebook as tqdm

### Functions

In [33]:
# compute sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# numerical way to compute gradient or jacobian of vector function
h = 1e-6
def gradient(f):
    def grad_f(x):
        if isinstance(x, list):
            x = np.array(x)
        I = np.eye(x.shape[0])
        return np.array([(f(x + h * I_) - f(x - h * I_)) / (2.0 * h) for I_ in I])
    return grad_f

# preprocessed wrapper of the polynomial transform
class PolinomialTransform:
    def __init__(self, degree=10):
        self.degree = degree
        self.obj = PolynomialFeatures(degree=self.degree)
        
    def __call__(self, x):
        return self.obj.fit_transform([x]).reshape(-1,)

### Data loading

In [4]:
links = pd.read_csv('data/small/links.csv')
movies = pd.read_csv('data/small/movies.csv')
ratings = pd.read_csv('data/small/ratings.csv')
tags = pd.read_csv('data/small/tags.csv')

### User-item matrix creation

In [5]:
user_item = ratings.pivot(index='userId', columns='movieId', values='rating')

### Data cleaning

In [6]:
user_item = ratings.pivot(index='userId', columns='movieId', values='rating')

In [7]:
print('Exist {x} unique films with {y} unique users rated these films.'.format(y=ratings.userId.unique().shape[0],
                                                                              x=movies.movieId.unique().shape[0]))

Exist 9742 unique films with 610 unique users rated these films.


In [8]:
rated_films_differences = len(set(movies.movieId.unique()) - set(user_item.columns))

print('Users did not rate {x} films, so we will delete it from user-item matrix,\
 because we will have column with only NaNs.'.format(x=rated_films_differences))

print('Each user ranked only a few films at all, so our matrix will have a lot of NaNs. User-item matrix has {x} \
dimension, hence {y} overall ratings. But {z} of them are empty, so we have {t:.1f}% existed ratings.'
      .format(x=user_item.shape,
              y=user_item.shape[0] * user_item.shape[1],
              z=user_item.isna().sum().sum(),
              t=100 * (1 - user_item.isna().sum().sum() / (user_item.shape[0] * user_item.shape[1]))))

Users did not rate 18 films, so we will delete it from user-item matrix, because we will have column with only NaNs.
Each user ranked only a few films at all, so our matrix will have a lot of NaNs. User-item matrix has (610, 9724) dimension, hence 5931640 overall ratings. But 5830804 of them are empty, so we have 1.7% existed ratings.


### SVD

In [9]:
class VanillaSVD(AlgoBase):

    def __init__(self, n_factors=10, n_epochs=10, lr=4e-3, reg_size=0.1):
        AlgoBase.__init__(self)
        
        self.n_factors = n_factors
        self.n_epochs = n_epochs
        self.lr = lr
        self.reg_size = reg_size
        
    def fit(self, trainset):
        
        AlgoBase.fit(self, trainset)
        self.sgd(trainset)

        return self
        
    def sgd(self, trainset):
        
        self.trainset = trainset
        
        self.p = np.random.normal(0.0, 0.1, (self.trainset.n_users, self.n_factors))
        self.q = np.random.normal(0.0, 0.1, (self.trainset.n_items, self.n_factors))
        
        self.n_ratings = self.trainset.n_ratings
        
        for current_epoch in tqdm(range(self.n_epochs)):
            
            for u, i, r in self.trainset.all_ratings():
                
                error = r - self.p[u].dot(self.q[i])
                    
                self.p[u] += self.lr * (error * self.q[i] - self.reg_size * self.p[u])
                self.q[i] += self.lr * (error * self.p[u] - self.reg_size * self.q[i])
                    
    def estimate(self, u, i):
    
        known_user = self.trainset.knows_user(u)
        known_item = self.trainset.knows_item(i)
        
        if known_user and known_item:
            return np.dot(self.p[u], self.q[i])
        else:
            raise PredictionImpossible('User or item are unknown')

### Probabilistic Matrix Factorization

In [13]:
class VanillaPMF(AlgoBase):

    def __init__(self, n_factors=10, n_epochs=10, lr=4e-3, sigma_p=0.1, sigma_q=0.1):
        AlgoBase.__init__(self)
        
        self.n_factors = n_factors
        self.n_epochs = n_epochs
        self.lr = lr
        
        self.sigma_p = sigma_p
        self.sigma_q = sigma_q
        
        self.reg_size_p = 0.001 / self.sigma_p
        self.reg_size_q = 0.001 / self.sigma_q
        
    def fit(self, trainset):
        
        AlgoBase.fit(self, trainset)
        self.sgd(trainset)

        return self
        
    def sgd(self, trainset):
        
        self.trainset = trainset
        
        self.p = np.random.normal(0.0, self.sigma_p, (self.trainset.n_users, self.n_factors))
        self.q = np.random.normal(0.0, self.sigma_q, (self.trainset.n_items, self.n_factors))
        
        self.n_ratings = self.trainset.n_ratings
        
        for current_epoch in tqdm(range(self.n_epochs)):
            
            for u, i, r in self.trainset.all_ratings():
                
                scaled_r = (r - 1)/4
                sigm = sigmoid(self.p[u].dot(self.q[i]))
                error = scaled_r - sigm
                    
                self.p[u] += self.lr * (error * sigm * (1 - sigm) * self.q[i] - self.reg_size_p * self.p[u])
                self.q[i] += self.lr * (error * sigm * (1 - sigm) * self.p[u] - self.reg_size_q * self.q[i])
                    
    def estimate(self, u, i):
    
        known_user = self.trainset.knows_user(u)
        known_item = self.trainset.knows_item(i)
        
        if known_user and known_item:
            return 1 + 4*sigmoid(np.dot(self.p[u], self.q[i]))
        else:
            raise PredictionImpossible('User or item are unknown')

### Evaluation of ours own implementations

In [20]:
# load data in specified format
data = Dataset.load_builtin('ml-100k')

# create an instance of PMF
svd = VanillaSVD(n_factors=10, n_epochs=20, lr=0.005, reg_size=0.02)
pmf = VanillaPMF(n_factors=10, n_epochs=1000, lr=0.005, sigma_p=0.1, sigma_q=0.1)

In [21]:
# svd
cross_validate(svd, data, verbose=True)


Evaluating RMSE, MAE of algorithm VanillaSVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9412  0.9507  0.9379  0.9490  0.9494  0.9457  0.0051  
MAE (testset)     0.7367  0.7487  0.7377  0.7449  0.7465  0.7429  0.0048  
Fit time          26.61   26.69   27.23   33.51   26.14   28.03   2.76    
Test time         0.24    0.18    0.17    0.23    0.16    0.20    0.03    


{'test_rmse': array([0.94122397, 0.95068277, 0.9379408 , 0.94900206, 0.94944782]),
 'test_mae': array([0.73672358, 0.7486805 , 0.73774508, 0.74487183, 0.74646811]),
 'fit_time': (26.60607933998108,
  26.687235116958618,
  27.234275817871094,
  33.510265827178955,
  26.135029077529907),
 'test_time': (0.2411501407623291,
  0.17968106269836426,
  0.16622591018676758,
  0.23294687271118164,
  0.15787196159362793)}

In [22]:
# pmf
cross_validate(pmf, data, verbose=True)


Evaluating RMSE, MAE of algorithm VanillaPMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9552  0.9620  0.9641  0.9592  0.9659  0.9613  0.0038  
MAE (testset)     0.7742  0.7787  0.7819  0.7766  0.7796  0.7782  0.0026  
Fit time          1275.56 1620.69 2797.37 1462.35 1272.56 1685.71 570.76  
Test time         0.49    0.31    0.72    0.39    0.17    0.42    0.18    


{'test_rmse': array([0.95516462, 0.96204901, 0.96407844, 0.95920772, 0.96590861]),
 'test_mae': array([0.7741997 , 0.77871775, 0.78185539, 0.77660587, 0.77962201]),
 'fit_time': (1275.5576498508453,
  1620.6908457279205,
  2797.3733971118927,
  1462.3481562137604,
  1272.560527086258),
 'test_time': (0.4883239269256592,
  0.3146672248840332,
  0.7196810245513916,
  0.3867766857147217,
  0.17207813262939453)}

### Evaluation of implementations from Surprise

In [23]:
surp_svd = SVD(n_factors=10, n_epochs=20, lr_all=0.005, reg_all=0.02)
surp_pmf = SVD(n_factors=10, n_epochs=100, lr_all=0.005, biased=False)

In [24]:
cross_validate(surp_svd, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9377  0.9350  0.9477  0.9258  0.9355  0.9363  0.0070  
MAE (testset)     0.7405  0.7375  0.7473  0.7314  0.7392  0.7392  0.0051  
Fit time          1.99    1.86    1.86    2.17    2.29    2.03    0.17    
Test time         0.23    0.14    0.14    0.22    0.16    0.18    0.04    


{'test_rmse': array([0.93769449, 0.93495498, 0.94765816, 0.92579726, 0.93545862]),
 'test_mae': array([0.74048312, 0.73746301, 0.74731858, 0.73140319, 0.7392106 ]),
 'fit_time': (1.9911041259765625,
  1.8611950874328613,
  1.8561208248138428,
  2.1724789142608643,
  2.2917239665985107),
 'test_time': (0.2286241054534912,
  0.14422106742858887,
  0.14364194869995117,
  0.21912789344787598,
  0.15707921981811523)}

In [25]:
cross_validate(surp_pmf, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9641  0.9657  0.9637  0.9683  0.9718  0.9667  0.0030  
MAE (testset)     0.7466  0.7458  0.7457  0.7536  0.7538  0.7491  0.0038  
Fit time          9.94    9.61    10.14   10.55   10.85   10.22   0.44    
Test time         0.21    0.21    0.12    0.26    0.13    0.19    0.05    


{'test_rmse': array([0.96410441, 0.96570176, 0.96367723, 0.96833408, 0.97175101]),
 'test_mae': array([0.74656017, 0.74578546, 0.74572985, 0.75356606, 0.75381233]),
 'fit_time': (9.937514066696167,
  9.607587099075317,
  10.135118961334229,
  10.547491073608398,
  10.853492975234985),
 'test_time': (0.21497201919555664,
  0.20709896087646484,
  0.12357306480407715,
  0.2603778839111328,
  0.12807798385620117)}

### Kernel SVD implementation

In [34]:
transform = PolinomialTransform(degree=2)

In [35]:
class KernelSVD(AlgoBase):

    def __init__(self, transform, n_factors=3, n_epochs=20, lr=2e-3, reg_size=1e-5):
        AlgoBase.__init__(self)
        
        self.n_factors = n_factors
        self.n_epochs = n_epochs
        self.lr = lr
        self.reg_size = reg_size
        self.transform = transform
        self.jacobian = gradient(self.transform)
        
    def fit(self, trainset):
        
        AlgoBase.fit(self, trainset)
        self.sgd(trainset)

        return self
        
    def sgd(self, trainset):
        
        self.trainset = trainset
        
        self.p = np.random.normal(0.0, 0.1, (self.trainset.n_users, self.n_factors))
        self.q = np.random.normal(0.0, 0.1, (self.trainset.n_items, self.n_factors))
        
        self.n_ratings = self.trainset.n_ratings
        
        for current_epoch in range(self.n_epochs):
                        
            for u, i, r in self.trainset.all_ratings():
                
                tr_users_vector = self.transform(self.p[u])
                tr_items_vector = self.transform(self.q[i])
                                
                error = r - np.dot(tr_users_vector, tr_items_vector)
                
                self.p[u] += self.lr * error * self.jacobian(self.p[u]) @ tr_items_vector 
                self.q[i] += self.lr * error * self.jacobian(self.q[i]) @ tr_users_vector 
                    
    def estimate(self, u, i):
    
        known_user = self.trainset.knows_user(u)
        known_item = self.trainset.knows_item(i)
        
        if known_user and known_item:
            return np.dot(self.transform(self.p[u]), self.transform(self.q[i]))
        else:
            raise PredictionImpossible('User or item are unknown')

data = Dataset.load_builtin('ml-100k')
transform = PolinomialTrannsform(degree=2)
algo = KernelSVD(transform=transform)

In [47]:
_ = cross_validate(algo, data, verbose=True)

Evaluating RMSE, MAE of algorithm KernelSVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9919  0.9733  0.9867  0.9789  0.9776  0.9817  0.0067  
MAE (testset)     0.7752  0.7632  0.7702  0.7658  0.7639  0.7677  0.0045  
Fit time          3099.40 3057.05 3219.62 3248.54 3248.67 3174.66 80.56   
Test time         5.33    5.52    5.67    5.75    5.72    5.60    0.16    
